In [1]:
print(spark.version)

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5,application_1752831585852_0006,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3.5.5-amzn-0

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Taxi Analysis").getOrCreate()


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1752831585852_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, LongType
import pyspark.sql.functions as F

YELLOW_TAXI_PATH = f"s3://robot-dreams-source-data/home-work-1-unified/nyc_taxi/yellow/"
GREEN_TAXI_PATH = f"s3://robot-dreams-source-data/home-work-1-unified/nyc_taxi/green/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql.functions import col, lit
from pyspark.sql.types import DoubleType, IntegerType

# Чтение parquet-файлов
yellow_df = spark.read.option("recursiveFileLookup", "true").parquet(YELLOW_TAXI_PATH)
green_df =  spark.read.option("recursiveFileLookup", "true").parquet(GREEN_TAXI_PATH)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
green_df.printSchema()
yellow_df.printSchema()
green_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- VendorID: long (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- taxi_type: string (nullable = false)

root
 |-- VendorID: long (nullable = true)
 |-- passenger_count: long (nullable = true)
 |--

In [6]:
# Добавление колонки типа такси
yellow_df = yellow_df.withColumn("taxi_type", lit("yellow"))
green_df = green_df.withColumn("taxi_type", lit("green"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Объединеняем два dataFrame
raw_trips_df = yellow_df.unionByName(green_df)

raw_trips_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------------+---------------------+-----------+---------+
|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|tpep_pickup_datetime|tpep_dropoff_datetime|Airport_fee|taxi_type|
+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------------+---------------------+-----------+---------+
|       1|              1|          0.0|         1|                 N|         140|         140|           2|        2.5|  0.5|    0.5|       0.0|         0.

In [10]:
from pyspark.sql.functions import col, unix_timestamp

# Добавляем колонку - продолжительность поездки
raw_trips_df = raw_trips_df.withColumn(
    "trip_duration_min",
    (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60
)

# Фильтры
filtered_trips_df = raw_trips_df.filter(
    (col("trip_distance") >= 0.1) &
    (col("fare_amount") >= 2.0) &
    (col("trip_duration_min") >= 1)
)

filtered_trips_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------------+---------------------+-----------+---------+------------------+
|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|tpep_pickup_datetime|tpep_dropoff_datetime|Airport_fee|taxi_type| trip_duration_min|
+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+--------------------+---------------------+-----------+---------+------------------+
|       1|              1|          2.0|         1|                 N|         107|         249|    

In [12]:
from pyspark.sql.functions import hour, date_format, unix_timestamp, col

# Добавляем колонку pickup_hour (час начала поездки)
filtered_trips_df = filtered_trips_df.withColumn("pickup_hour", hour(col("tpep_pickup_datetime")))

# Добавляем колонку pickup_day_of_week (день недели: Monday, Tuesday, …)
filtered_trips_df = filtered_trips_df.withColumn("pickup_day_of_week", date_format(col("tpep_pickup_datetime"), "EEEE"))

# Добавляем колонку duration_min (продолжительность в минутах)
filtered_trips_df = filtered_trips_df.withColumn(
    "duration_min",
    (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60
)

filtered_trips_df.select("tpep_pickup_datetime", "pickup_hour", "pickup_day_of_week", "duration_min").show(10, truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+------------------+------------------+
|tpep_pickup_datetime|pickup_hour|pickup_day_of_week|duration_min      |
+--------------------+-----------+------------------+------------------+
|2014-02-01 00:05:03 |0          |Saturday          |10.683333333333334|
|2014-02-01 00:17:19 |0          |Saturday          |2.933333333333333 |
|2014-02-01 00:24:08 |0          |Saturday          |12.983333333333333|
|2014-02-01 00:00:23 |0          |Saturday          |9.05              |
|2014-02-01 00:18:58 |0          |Saturday          |3.6               |
|2014-02-01 00:24:05 |0          |Saturday          |32.06666666666667 |
|2014-02-01 00:08:11 |0          |Saturday          |26.0              |
|2014-02-01 00:51:34 |0          |Saturday          |22.05             |
|2014-02-01 00:06:00 |0          |Saturday          |5.466666666666667 |
|2014-02-01 00:29:42 |0          |Saturday          |7.333333333333333 |
+--------------------+-----------+-----------------

In [13]:
# считываем taxi_zone_lookup.csv для соединения с датафреймом
taxi_zones_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("s3://robot-dreams-source-data/home-work-1-unified/nyc_taxi/taxi_zone_lookup.csv")

#taxi_zones_df.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# JOIN для pickup_zone
filtered_trips_df = filtered_trips_df.join(
    taxi_zones_df.withColumnRenamed("LocationID", "PULocationID").withColumnRenamed("Zone", "pickup_zone"),
    on="PULocationID",
    how="left"
)

# JOIN для dropoff_zone
filtered_trips_df = filtered_trips_df.join(
    taxi_zones_df.withColumnRenamed("LocationID", "DOLocationID").withColumnRenamed("Zone", "dropoff_zone"),
    on="DOLocationID",
    how="left"
)

filtered_trips_df.select("PULocationID", "pickup_zone", "DOLocationID", "dropoff_zone").show(10, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----------------------------+------------+-------------------------+
|PULocationID|pickup_zone                  |DOLocationID|dropoff_zone             |
+------------+-----------------------------+------------+-------------------------+
|107         |Gramercy                     |249         |West Village             |
|158         |Meatpacking/West Village West|90          |Flatiron                 |
|234         |Union Sq                     |246         |West Chelsea/Hudson Yards|
|107         |Gramercy                     |140         |Lenox Hill East          |
|162         |Midtown East                 |141         |Lenox Hill West          |
|141         |Lenox Hill West              |112         |Greenpoint               |
|237         |Upper East Side South        |87          |Financial District North |
|211         |SoHo                         |239         |Upper West Side South    |
|246         |West Chelsea/Hudson Yards    |48          |Clinton East       

In [ ]:
#df_with_zones.select("PULlocationID", "pickup_zone", "DOlocationID", "dropoff_zone").show(5, truncate=False)


In [15]:
from pyspark.sql.functions import col, count, avg, max, min, sum as spark_sum, when

# Общее кол-во поездок в зоне
zone_summary = filtered_trips_df.groupBy("pickup_zone").agg(
    count("*").alias("total_trips"),
    avg("trip_distance").alias("avg_trip_distance"),
    avg("total_amount").alias("avg_total_amount"),
    avg("tip_amount").alias("avg_tip_amount"),
    spark_sum(when(col("taxi_type") == "yellow", 1).otherwise(0)).alias("yellow_trips"),
    spark_sum(when(col("taxi_type") == "green", 1).otherwise(0)).alias("green_trips"),
    max("trip_distance").alias("max_trip_distance"),
    min("tip_amount").alias("min_tip_amount")
)

# часть зеленых и желтых поездок
zone_summary = zone_summary.withColumn("yellow_share", col("yellow_trips") / col("total_trips")) \
                           .withColumn("green_share", col("green_trips") / col("total_trips")) \
                           .drop("yellow_trips", "green_trips")

zone_summary.show(truncate=False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+-----------+------------------+------------------+------------------+-----------------+--------------+------------+-----------+
|pickup_zone                   |total_trips|avg_trip_distance |avg_total_amount  |avg_tip_amount    |max_trip_distance|min_tip_amount|yellow_share|green_share|
+------------------------------+-----------+------------------+------------------+------------------+-----------------+--------------+------------+-----------+
|Pelham Parkway                |9879       |5.780610385666564 |25.234409353173394|1.185459054560178 |104.15           |0.0           |1.0         |0.0        |
|Rego Park                     |70131      |5.716086324164775 |24.61390141306976 |2.012534257318447 |46532.54         |0.0           |1.0         |0.0        |
|Kew Gardens Hills             |18556      |9.904392110368615 |26.047876697564142|1.5367821728820863|68704.94         |0.0           |1.0         |0.0        |
|Jackson Heights               |403558  

In [16]:
from datetime import datetime

# Подготовим дату и путь
today_str = datetime.today().strftime('%Y-%m-%d')

output_path = f"s3://izabelkin-emr-studio/results/zone_statistic/{today_str}/zone_statistic.parquet"

# Сохраняем в один parquet-файл
zone_summary \
    .coalesce(1) \
    .write \
    .mode("overwrite") \
    .parquet(output_path)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
from pyspark.sql.functions import count, sum as _sum, when, col, to_date
from datetime import datetime

# Агрегация по зонам и дням недели
zone_day_stats = filtered_trips_df.groupBy("pickup_zone", "pickup_day_of_week").agg(
    count("*").alias("total_trips"),
    _sum(when(col("total_amount") > 30, 1).otherwise(0)).alias("high_fare_trips")
)

# Расчет дорогих поездок 
zone_day_stats = zone_day_stats.withColumn(
    "high_fare_share", col("high_fare_trips") / col("total_trips")
).drop("high_fare_trips")

# Формування шляху до S3 з датою
today_str = datetime.today().strftime("%Y-%m-%d")
output_path = f"s3://izabelkin-emr-studio/results/zone_days_statstic/{today_str}/zone_days_statstic.parquet"

# Запис у S3 (одним файлом)
zone_day_stats.coalesce(1).write.mode("overwrite").parquet(output_path)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…